In [1022]:
import numpy as np
import pandas as pd
import re
import urllib
import json
import contextlib, unicodedata, sys
import matplotlib.pyplot as plt
import torch
import difflib
from difflib import SequenceMatcher
import os

In [1023]:
def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif

In [1024]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [647]:
%cd /Users/quert/Downloads/gcp_tmp/raw_text
clusters = []
with open("title_page_whole_test.txt", "r") as fread:
    for line in fread.readlines():
        clusters.append(json.loads(line.strip()))
%cd /Users/quert/Downloads/gcp_tmp/

/Users/quert/Downloads/gcp_tmp/raw_text
/Users/quert/Downloads/gcp_tmp


In [ ]:
clusters[0].keys()
# title_non_update_link
# title_update_link
# update_bs4
# non_update_bs4

### Get the indices of required instances
* Keep English language, with non-updated full-content existed, and meaningful updated full-content

In [648]:
# create idx list
all_idx = [i for i in range(len(clusters))]
rmved_idx = []

for idx in range(len(clusters)):
    try:
        if bool(len(clusters[idx]['non_update_all_paragraph'].split('. ')) == 1):
            rmved_idx.append(idx)
    except:
        error_idx.append(idx)
idx_list = Diff(all_idx, rmved_idx)
len(all_idx), len(idx_list)

(239, 232)

In [649]:
rmved_idx = []
for idx in idx_list:
    if len(clusters[idx]['update_first_paragraph'].split())<=10:
        rmved_idx.append(idx)
id_list = Diff(idx_list, rmved_idx)
len(idx_list), len(id_list)

(232, 226)

In [650]:
# filter by language of summ==English
en_ids = []
for idx in id_list:
    if isEnglish(clusters[idx]['wiki_portal_summary']):
        en_ids.append(idx)
len(id_list), len(en_ids)
# after filtering, we have 201 instances in test set

(226, 201)

In [587]:
len(sec_ids)

152

In [ ]:
# convert id with same section name to dataframe
# train_sec_df = pd.DataFrame({'Global ID': sec_ids}).to_csv('./train_sec.csv')
# test_sec_df = pd.DataFrame({'Global ID': sec_ids}).to_csv('./test_sec.csv')
# val_sec_df = pd.DataFrame({'Global ID': sec_ids}).to_csv('./val_sec.csv')

### Parse the unicode into text
* Parse the unicode into text
* Paragraphs are separate with `. \\c\\c`

In [ ]:
log = []
with open("val_text.txt.src", "w") as f:
    # for idx in en_ids:
    for idx in sec_ids:
        # bef_rec = []
        non_update_all_paragraph = clusters[idx]['non_update_all_paragraph'].split('\n')
        # bef_rec.append(non_update_all_paragraph)
        s = '.\c'
        bef_rec_str = s.join(non_update_all_paragraph).replace("\\\\\c", "\c\c")
        with contextlib.redirect_stdout(f):
            print({unicodedata.normalize('NFKD', bef_rec_str).encode('utf-8', 'ignore').decode('utf-8')})
with open("val_text.txt.src", "r") as f:
    for line in f.readlines():
        log.append(line.strip().replace(".\\\\c", "\c\c").replace("\\\'s", "'s").replace("{\'", "").replace("\'}", "").replace(".\\c\\c", ". \\c\\c"))
with open("val_text.txt.src", "w") as f:
    for line in log:
        f.write(line+"\n")

In [ ]:
log = []
with open("val_text.txt.tgt", "w") as f:
    # for idx in en_ids:
    for idx in sec_ids:
        # tgt_rec = []
        update_all_paragraph = clusters[idx]["update_all_paragraph"].split("\n")
        # tgt_rec.extend(update_all_paragraph)
        s = ".\c"
        tgt_rec_str = s.join(update_all_paragraph).replace("\\\\\c", "\c\c")
        with contextlib.redirect_stdout(f):
            print({unicodedata.normalize('NFKD', tgt_rec_str).encode('utf-8', 'ignore').decode('utf-8')})
with open("val_text.txt.tgt", "r") as f:
    for line in f.readlines():
        log.append(line.strip().replace(".\\\\c", "\c\c").replace("\\\'s", "'s").replace("{\'", "").replace("\'}", "").replace(".\\c\\c", ". \\c\\c"))
with open("val_text.txt.tgt", "w") as f:
    for line in log:
        f.write(line+"\n")

In [ ]:
# Convert the text (src, tgt) to pt format
srcs, tgts = [], []
with open("val_text.txt.src", "r") as f:
    for line in f.readlines():
        srcs.append(line.strip())
with open("val_text.txt.tgt", "r") as f:
    for line in f.readlines():
        tgts.append(line.strip())

wrap = []
for idx in range(len(srcs)):
    idx_content = {}
    idx_content['document'] = srcs[idx]
    idx_content['summary'] = tgts[idx].lstrip('{"').lstrip("{'").rstrip('\n')
    wrap.append(idx_content)
torch.save(wrap, 'netku_samesecs_val.pt')

### Extract the section name and number of paragraphs under each section

In [633]:
def Old_pars_num(clu_idx):
    nonupdated_raw_text = clusters[clu_idx]["non_update_bs4"]
    # end_idx = re.findall('<span class="mw-headline" id=.+">(.*)</span>', raw_text).index("References")
    # sec_names = re.findall('<span class="toctext">(.*)</span>', raw_text)[:end_idx]
    
    old_main_secs = re.findall('<span class="tocnumber">\d+</span> <span class="toctext">(.*)</span>', nonupdated_raw_text)
    old_all_secs = re.findall('<span class="mw-headline" id=.+">(.*)</span>', nonupdated_raw_text)
    
    # Return the global index of main section
    old_main_global_ids = []
    for old_main_sec in old_main_secs:
        # The indices of main sec in all_secs
        ids = old_all_secs.index(old_main_sec)
        old_main_global_ids.append(ids)
    
    # Concatenate the parent section name with sub-section name
    # Some page only include summary without other sections
    old_mod_secs = old_all_secs.copy()
    try:
        for idx in range(len(old_mod_secs)):
            if idx in old_main_global_ids: old_mod_secs[idx]==old_mod_secs[idx]
            else: 
                old_bool_lst = list(np.asarray(old_main_global_ids)<idx)
                old_parent_ord = [str(val) for val in old_bool_lst].count("True")
                old_parent_idx = old_main_global_ids[old_parent_ord-1]
                old_mod_secs[idx] = str(old_mod_secs[old_parent_idx]) + " - " + old_all_secs[idx]
        
        # Get the section names except "References"
        old_end_idx = old_mod_secs.index("References")
        old_sec_names = old_mod_secs[:old_end_idx]
        
        # Count the #paragraphs for each section (main section and sub-section), exclude the section with zero paragraph 
        old_num_pars = []
        for i in range(len(old_sec_names)):
            old_pars_str = str(nonupdated_raw_text.split('<span class="mw-headline')[i+1])
            old_num_par = len(re.split("<p>|<li>", old_pars_str)[1:])
            # old_num_par = len(re.split("<p>", old_pars_str)[1:])
            old_num_pars.append(old_num_par)
        old_num_summ = len(clusters[clu_idx]["non_update_wiki_summary"].split("\n"))
        old_num_pars.insert(0, old_num_summ)
        old_sec_names.insert(0, "Summary")
        old_sec_pars = dict(zip(old_sec_names, old_num_pars))
        old_secs_pars_nums = {k: v for k, v in old_sec_pars.items() if v!=0}
        if "Notes" in old_secs_pars_nums.keys(): del old_secs_pars_nums["Notes"]
        elif "See also" in old_secs_pars_nums.keys(): del old_secs_pars_nums["See also"]
        
    except: # page with only "Summary" exists
        old_sec_names, old_num_pars = [], []
        old_num_summ = len(clusters[clu_idx]["non_update_wiki_summary"].split("\n"))
        old_sec_names.append("Summary")
        old_num_pars.append(old_num_summ) 
        old_sec_pars = dict(zip(old_sec_names, old_num_pars))
        old_secs_pars_nums = {k: v for k, v in old_sec_pars.items() if v!=0}
        
    # return list(old_secs_pars_nums)
    return list(old_secs_pars_nums.keys())
    # return old_secs_pars_nums

In [634]:
def New_pars_num(clu_idx):
    updated_raw_text = clusters[clu_idx]["update_bs4"] 
    
    new_main_secs = re.findall('<span class="tocnumber">\d+</span> <span class="toctext">(.*)</span>', updated_raw_text)
    new_all_secs = re.findall('<span class="mw-headline" id=.+">(.*)</span>', updated_raw_text)
    
    # Return the global index of main section
    new_main_global_ids = []
    for new_main_sec in new_main_secs:
        # The indices of main sec in all_secs
        ids = new_all_secs.index(new_main_sec)
        new_main_global_ids.append(ids)
    
    # Concatenate the parent section name with sub-section name
    new_mod_secs = new_all_secs.copy()
    try:
        for idx in range(len(new_mod_secs)):
            if idx in new_main_global_ids: new_mod_secs[idx]==new_mod_secs[idx]
            else: 
                new_bool_lst = list(np.asarray(new_main_global_ids)<idx)
                new_parent_ord = [str(val) for val in new_bool_lst].count("True")
                new_parent_idx = new_main_global_ids[new_parent_ord-1]
                new_mod_secs[idx] = str(new_mod_secs[new_parent_idx]) + " - " + new_all_secs[idx]
        
        # Get the section names except "References"
        new_end_idx = new_mod_secs.index("References")
        new_sec_names = new_mod_secs[:new_end_idx]
        
        # Count the #paragraphs under each section (main section and sub-section), exclude the section with zero paragraph 
        new_num_pars = []
        for i in range(len(new_sec_names)):
            new_pars_str = str(clusters[clu_idx]["update_bs4"].split('<span class="mw-headline')[i+1])
            new_num_par = len(re.split("<p>|<li>", new_pars_str)[1:])
            # new_num_par = len(re.split("<p>", new_pars_str)[1:])
            new_num_pars.append(new_num_par)
        new_num_summ = len(clusters[clu_idx]["non_update_wiki_summary"].split("\n"))
        new_num_pars.insert(0, new_num_summ)
        new_sec_names.insert(0, "Summary")
        new_sec_pars = dict(zip(new_sec_names, new_num_pars))
        new_secs_pars_nums = {k: v for k, v in new_sec_pars.items() if v!=0}
        
        if "Notes" in new_secs_pars_nums.keys(): del new_secs_pars_nums["Notes"]
        elif "See also" in new_secs_pars_nums.keys(): del new_secs_pars_nums["See also"]
        
    except:
        new_sec_names, new_num_pars = [], []
        new_num_summ = len(clusters[clu_idx]["non_update_wiki_summary"].split("\n"))
        new_sec_names.append("Summary")
        new_num_pars.append(new_num_summ) 
        new_sec_pars = dict(zip(new_sec_names, new_num_pars))
        new_secs_pars_nums = {k: v for k, v in new_sec_pars.items() if v!=0}
        
    # return list(new_secs_pars_nums)
    return list(new_secs_pars_nums.keys())
    # return new_secs_pars_nums

In [651]:
# extract the instances with same sections
sec_ids = []
for idx in en_ids:
    try:
        if Old_pars_num(idx)==New_pars_num(idx):
            sec_ids.append(idx)
    except:
        pass
# -- Train set
# We have 911 instances with same #sections and #paragraphs
# We have 1334 instances with same #sections -> 423 instances with same #sections and different #paragraphs
# Apply the sec_ids to extract and construct new datset
# -- Test set
# We have 168 instances with same #sections and #paragraphs
# We have 168 instances with same #sections -> 0 instances with same #sections and different #paragraphs
# -- Val set
# We have instances with same #sections and #paragraphs
# We have instances with same #sections -> instances with same #sections and different #paragraphs

In [652]:
len(sec_ids)

168

In [ ]:
# Import our unsized data
%cd /Users/quert/Downloads/gcp_tmp
srcs, tgts = [], []
test_pt = torch.load("./ptfile/same_secs_old/netku_samesecs_train.pt")
for idx in range(len(test_pt)):
    srcs.append(test_pt[idx]["document"])
    tgts.append(test_pt[idx]["summary"])


In [ ]:
# Resize the paragraphs
new_srcs, new_tgts = [], []
for idx in range(len(srcs)):
    pars = srcs[idx].split("\\c\\c")
    nums = np.sum(list(Old_pars_num(sec_ids[idx]).values()))
    new_src = "\\c\\c".join(pars[:nums])
    new_srcs.append(new_src)
for idx in range(len(tgts)):
    pars = tgts[idx].split("\\c\\c")
    nums = np.sum(list(New_pars_num(sec_ids[idx]).values()))
    new_tgt = "\\c\\c".join(pars[:nums])
    new_tgts.append(new_tgt)

In [ ]:
# Create re-sized data
%cd /Users/quert/Downloads/gcp_tmp/ptfile/same_secs_new
with open("train_text.txt.src", "w") as f:
    for new_src in new_srcs:
        f.write(new_src+"\n")
with open("train_text.txt.tgt", "w") as f:
    for new_tgt in new_tgts:
        f.write(new_tgt+"\n")
        

In [ ]:
# Convert src, tgt to pt format
%cd /Users/quert/Downloads/gcp_tmp/ptfile/same_secs_new
wrap = []
for idx in range(len(new_srcs)):
    idx_content = {}
    idx_content['document'] = new_srcs[idx]
    idx_content['summary'] = new_tgts[idx]
    wrap.append(idx_content)
torch.save(wrap, 'netku_samesecs_train.pt')

### Insert section names to each paragraph

* Check if the #paragraphs from instances equals to our counts
* `. \\c\\c` -> `. <Timeline> \\c\\c`

In [615]:
# Store the numbers of each section name (dicts) into (list) 
old_list_secnums = [Old_pars_num(idx) for idx in sec_ids]
new_list_secnums = [New_pars_num(idx) for idx in sec_ids]

In [616]:
# Import the labeled data
srcs, tgts = [], []
test_pt = torch.load("./ptfile/same_secs_new/netku_samesecs_train.pt")
for idx in range(len(test_pt)):
    srcs.append(test_pt[idx]["document"])
    tgts.append(test_pt[idx]["summary"])


In [ ]:
# np.sum(list(old_list_secnums[0].values())), len(srcs[0].split("\\c\\c"))
# new_srcs = []
'''
nums = list(old_list_secnums[0].values())
titles = list(old_list_secnums[0].keys())
src_article = srcs[0]
paragraphs = src_article.split("\\c\\c")
paragraphs[0] = paragraphs[0]+title[0]
'''

In [620]:
# convert counts into indices: (index: title)
titles_keys = []
for idx in range(len(new_list_secnums)):
    titles = list(old_list_secnums[idx].keys())
    titles_as_keys = []
    for i in range(len(titles)):
        for _ in range(list(new_list_secnums[idx].values())[i]):
            titles_as_keys.append(titles[i])
    titles_keys.append(titles_as_keys)

In [621]:
# add section names to tail for each paragraph
added_secs = []
for idx in range(len(titles_keys)):
    splitted_pars = tgts[idx].split("\\c\\c")
    pars_in = []
    for i in range(len(splitted_pars)):
        par_with_secname = splitted_pars[i] + " <" + titles_keys[idx][i] + ">" + " \\c\\c"
        par_with_secname = par_with_secname.replace("  <", " <")
        pars_in.append(par_with_secname)
    pars = "".join(pars_in)
    added_secs.append(pars)

In [622]:
# Convert our new-construct data into src, tgt
%cd /Users/quert/Downloads/gcp_tmp/ptfile/same_secs_insert/ 
with open("train_text.txt.tgt", "w") as f:
    for instance in added_secs:
        f.write(instance+"\n")


/Users/quert/Downloads/gcp_tmp/ptfile/same_secs_insert


In [625]:
# Convert src, tgt into single pt format
%cd /Users/quert/Downloads/gcp_tmp/ptfile/same_secs_insert/
srcs, tgts = [], []
with open("train_text.txt.src", "r") as f:
    for line in f.readlines():
        srcs.append(line.strip())
with open("train_text.txt.tgt", "r") as f:
    for line in f.readlines():
        tgts.append(line.strip())

wrap = []
for idx in range(len(srcs)):
    idx_content = {}
    idx_content['document'] = srcs[idx]
    idx_content['summary'] = tgts[idx]
    wrap.append(idx_content)
torch.save(wrap, 'netku_samesecs_train.pt')

/Users/quert/Downloads/gcp_tmp/ptfile/same_secs_insert


### Trigger Extraction

In [653]:
%cd /Users/quert/Downloads/gcp_tmp/ptfile/same_secs_insert/
triggers = [clusters[idx]["wiki_portal_summary"] for idx in sec_ids]
with open("samesecs_triggers_test.txt", "w") as f:
    for trigger in triggers:
        f.write(trigger+"\n")

/Users/quert/Downloads/gcp_tmp/ptfile/same_secs_insert


### Extract paragraphs from each instance, and label as edited 0/1

In [1103]:
%cd /Users/quert/Downloads/gcp_tmp/ptfile/same_secs_insert_labeled/
val_pt = torch.load("val_samesecs_labeled.pt")

/Users/quert/Downloads/gcp_tmp/ptfile/same_secs_insert_labeled


In [1104]:
# Calculate the maen modification from each instance, and labeled 1 if bigger than mean value, else 1.
# Concate the paragraphs with its respective trigger
# Calculate the mean editions for each instance
mean_editions = []
for idx in range(len(val_pt)):
    for par in val_pt[idx]["summary"].split("\\c\\c"):
        editions = []
        num_editions = len(re.findall(r"\[ADD]", par)) + len(re.findall(r"\[SUB]", par)) + len(re.findall(r"\[RM]", par))
        editions.append(num_editions)
    mean_editions.append(np.mean(editions))

labels = []
for idx in range(len(val_pt)):
    for par in val_pt[idx]["summary"].split("\\c\\c"):
        if (len(re.findall(r"\[ADD]", par)) + len(re.findall(r"\[SUB]", par)) + len(re.findall(r"\[RM]", par))) > np.mean(mean_editions[idx]): labels.append(1)
        else: labels.append(0)


In [1105]:
# col[0]: paragraph + section name
col_0 = []
for idx in range(len(val_pt)):
    for par in val_pt[idx]["summary"].split("\\c\\c"):
    # for par in val_pt[idx]["document"].split("\\c\\c"):
        col_0.append(par)
    
# col[1]: trigger
triggers, col_1 = [], []
with open("samesecs_triggers_val.txt", "r") as f:
    for trigger in f.readlines():
        triggers.append(trigger)
for idx in range(len(val_pt)):
    for par in val_pt[idx]["summary"].split("\\c\\c"):
        col_1.append(triggers[idx])

# col[2]: target (label)
col_2 = labels.copy()
assert len(col_0)==len(col_1)==len(col_2)

In [1106]:
# remove the sentences with [ADD] at head
old_par = col_0.copy()
instances_par = []
for idx in range(len(old_par)):
    splits = re.split(r"\[KEEP\]", old_par[idx])
    splits = [ele for ele in splits if ele]
    for split in splits:
        try:
            if split.split()[0] in ["[ADD]", "[SUB]"]: splits.remove(split)
        except:
            pass
    instance = ("".join(splits)).replace("  ", " ")
    instances_par.append(instance.strip())

In [1107]:
# convert cols into dataframe
assert len(instances_par)==len(col_1)==len(col_2)
df = pd.DataFrame()
df["paragraphs_secs"] = instances_par
df["trigger"] = col_1
df["target"] = col_2
df.to_csv("merged_val.csv", index=False)

In [1108]:
# concate the paragraphs and tirgger columns into sources
df = pd.read_csv("merged_val.csv")
concats = []
for idx in range(df.shape[0]):
    row = df.iloc[idx, :]
    concated = str(row["paragraphs_secs"]) + " -- " + str(row["trigger"])
    concats.append(concated)
assert len(concats)==df["target"].shape[0]
df_updated = pd.DataFrame()
df_updated["paragraph"] = concats
df_updated["target"] = df["target"].tolist()
df_updated.to_csv("merged_updated_val.csv", index=False)

In [1110]:
df_train = pd.read_csv("merged_updated_val.csv")
df_train.iloc[0, :]

paragraph    On October 8, 2020, the U.S. Federal Bureau of...
target                                                       0
Name: 0, dtype: object

### Match paragraphs to respective instance

In [910]:
%cd /Users/quert/Downloads/gcp_tmp/ptfile/same_secs_insert_labeled/
test_pt = torch.load("test_samesecs_labeled.pt")

/Users/quert/Downloads/gcp_tmp/ptfile/same_secs_insert_labeled


In [911]:
num_par = []
for idx in range(len(test_pt)):
    num_par.append(len(test_pt[idx]["summary"].split("\\c\\c")))

In [912]:
test_df = pd.DataFrame({"Num of pars": num_par}).to_csv("./merged_numpar_test.csv")

In [ ]:
df = pd.read_csv("merged_updated_test.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10425 entries, 0 to 10424
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   paragraph  10425 non-null  object
 1   target     10425 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 163.0+ KB


In [920]:
df_slice = df.iloc[:, 1]
df_slice

0        0
1        0
2        0
3        0
4        0
        ..
10420    0
10421    0
10422    0
10423    0
10424    0
Name: target, Length: 10425, dtype: int64

In [924]:
df_list = df_slice.to_list()
len(df_list)

10425

#### Match paragraphs to its respective instance

In [1135]:
sub = pd.read_csv("submission.csv")
test_csv = pd.read_csv("merged_updated_test.csv")
num_par = pd.read_csv("merged_numpar_test.csv")
par_nums = num_par.iloc[:, 1].to_list()
assert len(sub) == np.sum(num_par.iloc[:, 1].to_list()) == len(test_csv)

In [1121]:
# merge paragraphs to instances
instances = []
start = 0
# for num in par_nums:
for idx in range(len(par_nums)):
    end_idx = start_idx + par_nums[idx] - 1
    the_range = [start_idx, end_idx]
    start = end_idx + 1
    

10425

### Fix the unbalanced problem

In [1111]:
df = pd.read_csv("merged_updated_train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73846 entries, 0 to 73845
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   paragraph  73846 non-null  object
 1   target     73846 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [1113]:
target = df.iloc[:, 1].to_list()
true_idx = [i for i in range(len(target)) if target[i]==1]
false_idx = [i for i in range(len(target)) if target[i]==0]
len(true_idx), len(false_idx)

(1972, 71874)

In [1114]:
# we sample 3 times of false instances
from random import sample
resample_false_idx = sorted(sample(false_idx, 2000))

In [1115]:
# resample the false example
resample_par, resample_idx = [], []
for idx in resample_false_idx:
    resample_par.append(df.iloc[idx, 0])
    resample_idx.append(df.iloc[idx, 1])
# combline the false and true example
for idx in true_idx:
    resample_par.append(df.iloc[idx, 0])
    resample_idx.append(df.iloc[idx, 1])

In [1116]:
pd.DataFrame({"paragraph": resample_par, "target": resample_idx}).to_csv("resample_train.csv")

### Examine the submission file

In [973]:
df_submission = pd.read_csv("submission.csv")
df_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10425 entries, 0 to 10424
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   target  10425 non-null  int64
dtypes: int64(1)
memory usage: 81.6 KB


In [974]:
df_submission.target.value_counts()

0    10421
1        4
Name: target, dtype: int64

In [970]:
# check the test data targets
df_test = pd.read_csv("merged_updated_test.csv")
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10425 entries, 0 to 10424
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   paragraph  10425 non-null  object
 1   target     10425 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 163.0+ KB


In [971]:
df_test.target.value_counts()

0    10104
1      321
Name: target, dtype: int64